In [ ]:
//// import dependencies...

/*
"io.circe %% circe-core % 0.10.1",
    "io.circe %% circe-generic % 0.10.1",
    "io.circe %% circe-parser % 0.10.1",
    "io.github.azhur %% kafka-serde-circe % 0.4.0",
    "com.typesafe.akka %% akka-stream-kafka % 1.0.3"
    */


In [ ]:
val dataFile = System.getenv("HOME") + "/data/stream/crypto-stream.json"

In [ ]:
import akka.actor.ActorSystem
import akka.actor.{Actor, Props}
import akka.pattern.ask
import akka.util.ByteString
import akka.util.Timeout
import akka.stream.ActorMaterializer
import akka.stream.scaladsl._
import java.nio.file.Paths
import java.util.Properties

import org.apache.kafka.clients.producer.{KafkaProducer, ProducerRecord}
import org.apache.kafka.common.serialization.StringSerializer
import akka.kafka.ProducerSettings

import scala.concurrent.duration._

implicit val system = ActorSystem("system")
implicit val materializer = ActorMaterializer()

In [ ]:
import _root_.io.circe.parser.decode
import _root_.io.circe.{ Decoder, Encoder }

case class Tick(exchange: String, pair: String, timestamp: Long, price: Double, volume: Double)

object Tick {
  import _root_.io.circe.generic.semiauto._

  implicit lazy val encoder: Encoder[Tick] = deriveEncoder[Tick]
  implicit lazy val decoder: Decoder[Tick] = deriveDecoder[Tick]
}

In [ ]:
import scala.concurrent.duration._
import scala.language.postfixOps

class MsgDelayer extends Actor {
  import context._
  var time = -1L
  def receive = {
    case tick: Tick if (time < 0 ) => {
      time = tick.timestamp
      context.system.scheduler.scheduleOnce(0 milliseconds, sender, tick)
    }
    case tick: Tick => {
      val delay = tick.timestamp - time
      time = tick.timestamp
      context.system.scheduler.scheduleOnce(delay second, sender, tick)
    } 
  }
}
object MsgDelayer{
  def props() = Props(new MsgDelayer())
}

In [ ]:
val delayer = system.actorOf(MsgDelayer.props())
implicit val askTimeout = Timeout(500.seconds)

In [ ]:
val props = new Properties()
  props.put("bootstrap.servers", "192.168.58.111:9092")
  props.put("client.id", "ScalaProducerExample")
  props.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
  props.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val producer = new KafkaProducer[String, String](props)

In [ ]:
FileIO.fromPath(Paths.get(dataFile))
      .via(Framing.delimiter(ByteString("\n"), 256, true).map(_.utf8String))
      .map(s => (decode[Tick](s)).right.toOption.get)
      .filter(_.exchange == "Kraken")
      .mapAsync(1)(tick => (delayer ? tick).mapTo[Tick])
      .map(tick => Tick.encoder(tick).noSpaces)
      .map(value => new ProducerRecord[String, String]("test", value))
      .map(msg => producer.send(msg))
      .runWith(Sink.ignore)